## Loading (0)

In [1]:
import pandas as pd
import numpy as np 
import os
import json
import sqlite3
from datetime import datetime

---

### Function definitions

In [2]:
def path_files(path_directory):
    """
    Extracts all files names in a directory
    :param path_directory: path to the directory to extract files from
    :return: list of file paths in a given directory
    """
        
    directory = os.listdir(path_directory)
    json_files = []
    
    # Append all path files to list of json_files 

    for file in directory:
        if file.startswith('airlines-'):
            json_files.append(os.path.join(path_directory, file))

    return json_files

In [3]:
count = 0

def read_json(path_file: str):
    """
    Reads given json file, try to read entire file at once 
    :param path_files: path of the file
    :return: DataFrame of all tweets in given file
    """
    
    # Path counter
    
    global count
    print(str(count) + " " + path_file)
    count = count + 1
    
    # Try to read entire file at once, dtypes for tweet id's given to avoid JSON large number inaccuracy

    try:
        df_tweet_objects = pd.read_json(path_file, dtype={'id_str': object, 'in_reply_to_user_id_str': object, 
                                                          'in_reply_to_status_id_str': object, 'quoted_status_id_str': object})
    
    # When an error occurs, read file line by line 
    
    except ValueError:
        with open(path_file) as json_file:
            list_tweets = []
            for json_object in json_file:
                try:
                    tweet = json.loads(json_object)
                    list_tweets.append(tweet)
                except ValueError:
                    pass

            df_tweet_objects = pd.DataFrame(list_tweets)

    return df_tweet_objects.dropna(subset=['id_str', 'user'])

In [4]:
def tables(tweets):
    """
    Extracts nested dictionaries and
    :param tweets: DataFrame of all tweet information
    :return: one DataFrame for tweets and one DataFrame for users   
    """
    
    # Using pandas json_normalize to extract all keys from the nested user dictionary 
    
    users = pd.json_normalize(tweets['user'])
    
    # For loop to extract full_text from nested extended_tweet dictionary 

    for index, dictionary in enumerate(tweets['extended_tweet']):
        try:
            full_text = dictionary.get('full_text', np.nan)
            tweets.at[index, 'full_text'] = full_text
        except:
            pass
    
    # For loop to extract mentioned user id's from nested entities dictionary, return as tuple of user id's
    
    for index, dictionary in enumerate(tweets['entities']):
        mention_list = []
        for record in dictionary['user_mentions']:
            try:
                mentions = record.get('id_str', np.nan)
                mention_list.append(mentions)
            except:
                pass
        
        tweets.at[index, 'mentioned_users'] = ', '.join(filter(None, mention_list))
    
    tweets.rename(columns={'tweet_id': 'id_str'})

    # Match tweet and user id's to other tables, 'rename' columns and add empty columns as placeholders for later analysis
    
    tweets['tweet_id'] = tweets['id_str']        
    tweets['user_id'] = users['id_str']
    tweets['screen_name'] = users['screen_name']
    tweets['user_category'] = 'account'
    tweets['conversation_id'] = tweets['tweet_id']
    tweets['conversation_level'] = 0
    tweets['tokenized_text'] = 'unknown'
    tweets['created_at'] = pd.to_datetime(tweets['created_at'], infer_datetime_format=True)
    
    users['user_id'] = users['id_str']
    users['tweet_id'] = tweets['id_str']
    users['user_category'] = 'account'
    users['tweet_created_at'] = tweets['created_at']
    
    # Clean replies, quotes and tweet text by replacing nan values by proper values 
    
    tweets['reply_to_tweet'] = tweets['in_reply_to_status_id_str'].replace({np.nan: 0})
    tweets['reply_to_user'] = tweets['in_reply_to_user_id_str'].replace({np.nan: 0})
    tweets['quote_tweet'] = tweets['quoted_status_id_str'].replace({np.nan: 0})
    tweets['full_text'] = tweets['full_text'].fillna(tweets['text'])
    
    return tweets, users

In [5]:
def select(tweets, users):
    """
    Selects correct columns from DataFrames and cleans types
    :param tweets: DataFrame of all tweets
    :param users: DataFrame of all users
    :return: cleaned DataFrames 
    """
    
    # Copying useful columns into cleaned DataFrame 
    
    clean_tweets = tweets[['tweet_id', 'user_id', 'screen_name', 'user_category', 'full_text',
                           'tokenized_text', 'reply_to_tweet', 'reply_to_user', 'mentioned_users', 
                           'quote_tweet', 'conversation_id', 'conversation_level', 'lang', 'created_at']]
    
    clean_users = users[['user_id', 'tweet_id', 'name', 'screen_name', 'user_category', 'followers_count', 
                         'friends_count', 'statuses_count', 'verified', 'url', 'description', 'tweet_created_at']]
    
    # Defining correct types for columns in DataFrame 
    
    tweet_dict = {'tweet_id': int, 'user_id': int, 'screen_name': str, 'user_category': str, 'full_text': str, 'tokenized_text': str, 
                  'reply_to_tweet': int, 'reply_to_user': int, 'quote_tweet': int, 'mentioned_users': str, 
                  'conversation_id': str, 'conversation_level': int, 'lang': str, 'created_at': int}
    
    user_dict = {'user_id': int, 'tweet_id': int, 'name': str, 'screen_name': str, 'user_category': str,
                 'followers_count': int, 'friends_count': int, 'statuses_count': int, 'verified': bool, 
                 'url': str, 'description': str, 'tweet_created_at': int}
    
    clean_tweets = clean_tweets.astype(tweet_dict)
    clean_users = clean_users.astype(user_dict)
    
    # Adjusting created at columns to be in seconds 
    
    clean_tweets['created_at'] = clean_tweets['created_at'] // 1000000000
    clean_users['tweet_created_at'] = clean_users['tweet_created_at'] // 1000000000

    return clean_tweets, clean_users

In [6]:
def calculate_runtime(start, stop):
    """
    Calculates running time of every process. This cell is repeated in every notebook. 
    :param start: start time
    :param stop: stop time
    :return: output corresponding to running time for process 
    """
    
    runtime = stop - start
    
    days, seconds = runtime.days, runtime.seconds
    
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    
    return 'Runtime loading {} files was: {} h, {} m {} s'.format(count, hours, minutes, seconds)

---

### Loading json files into database

Using seperate folders allows for easier monitoring of the loading process

In [7]:
conn_tweets = sqlite3.connect('tweets_airlines.db')

---

In [8]:
start = datetime.now()

---

All files are loaded in batches to reduce running time. From batch, a dataframe is created. This dateframe is then normalized, all useless columns are deleted and every column is given the correct type. 

If it is preferred to run all files at ones, only run the cells corresponding to data folder 1. 

### Data Folder 1 

In [9]:
files = path_files('./data/')

In [10]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

0 ./data/airlines-1558744391657.json
1 ./data/airlines-1558888297881.json
2 ./data/airlines-1558527599826.json
3 ./data/airlines-1558779687636.json
4 ./data/airlines-1558697205154.json
5 ./data/airlines-1558863520888.json
6 ./data/airlines-1558974571041.json
7 ./data/airlines-1558611772040.json
8 ./data/airlines-1558623303180.json
9 ./data/airlines-1558546003827.json
10 ./data/airlines-1558678330070.json
11 ./data/airlines-1558998029487.json


In [11]:
df_tweets, df_users = tables(df_objects)

In [12]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [13]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="replace", index=False)

In [14]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="replace", index=False)

---

### Data Folder 2

In [15]:
files = path_files('./data_2/')

In [16]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

12 ./data_2/airlines-1559156713030.json
13 ./data_2/airlines-1559352498975.json
14 ./data_2/airlines-1559097885748.json
15 ./data_2/airlines-1559142170524.json
16 ./data_2/airlines-1559323725358.json
17 ./data_2/airlines-1559069822287.json
18 ./data_2/airlines-1559231904332.json
19 ./data_2/airlines-1559464285341.json
20 ./data_2/airlines-1559501630259.json
21 ./data_2/airlines-1559427581830.json
22 ./data_2/airlines-1559256655329.json


In [17]:
df_tweets, df_users = tables(df_objects)

In [18]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [19]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [20]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 3

In [21]:
files = path_files('./data_3/')

In [22]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

23 ./data_3/airlines-1559640222186.json
24 ./data_3/airlines-1559556316966.json
25 ./data_3/airlines-1559513326649.json
26 ./data_3/airlines-1559769009949.json
27 ./data_3/airlines-1559705236313.json
28 ./data_3/airlines-1559753967610.json
29 ./data_3/airlines-1559592585772.json
30 ./data_3/airlines-1559549054414.json
31 ./data_3/airlines-1559580503906.json
32 ./data_3/airlines-1559655706075.json
33 ./data_3/airlines-1559729883533.json


In [23]:
df_tweets, df_users = tables(df_objects)

In [24]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [25]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [26]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 4

In [27]:
files = path_files('./data_4/')

In [28]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

34 ./data_4/airlines-1559846508795.json
35 ./data_4/airlines-1559897122639.json
36 ./data_4/airlines-1559911151286.json
37 ./data_4/airlines-1559902121479.json
38 ./data_4/airlines-1559813570976.json
39 ./data_4/airlines-1559893511277.json
40 ./data_4/airlines-1559915161616.json
41 ./data_4/airlines-1559909155467.json
42 ./data_4/airlines-1559860041436.json
43 ./data_4/airlines-1559904272004.json
44 ./data_4/airlines-1559802437924.json


In [29]:
df_tweets, df_users = tables(df_objects)

In [30]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [31]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [32]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 5

In [33]:
files = path_files('./data_5/')

In [34]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

45 ./data_5/airlines-1559939261270.json
46 ./data_5/airlines-1559928540833.json
47 ./data_5/airlines-1559937665320.json
48 ./data_5/airlines-1559926788058.json
49 ./data_5/airlines-1559921160419.json
50 ./data_5/airlines-1559932564771.json
51 ./data_5/airlines-1559942816001.json
52 ./data_5/airlines-1559934119165.json
53 ./data_5/airlines-1559923002644.json
54 ./data_5/airlines-1559944147286.json
55 ./data_5/airlines-1559916992678.json


In [35]:
df_tweets, df_users = tables(df_objects)

In [36]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [37]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [38]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 6

In [39]:
files = path_files('./data_6/')

In [40]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

56 ./data_6/airlines-1559960873940.json
57 ./data_6/airlines-1559953866862.json
58 ./data_6/airlines-1559979261665.json
59 ./data_6/airlines-1559988476546.json
60 ./data_6/airlines-1559963157167.json
61 ./data_6/airlines-1559956008864.json
62 ./data_6/airlines-1559972279659.json
63 ./data_6/airlines-1559969180413.json
64 ./data_6/airlines-1559981995387.json
65 ./data_6/airlines-1559949250686.json
66 ./data_6/airlines-1559947622068.json


In [41]:
df_tweets, df_users = tables(df_objects)

In [42]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [43]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [44]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 7

In [45]:
files = path_files('./data_7/')

In [46]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

67 ./data_7/airlines-1559991535440.json
68 ./data_7/airlines-1560020476933.json
69 ./data_7/airlines-1560033347669.json
70 ./data_7/airlines-1560000901765.json
71 ./data_7/airlines-1560023189725.json
72 ./data_7/airlines-1559998347395.json
73 ./data_7/airlines-1560007831474.json
74 ./data_7/airlines-1560014686248.json
75 ./data_7/airlines-1560005733536.json
76 ./data_7/airlines-1560030119237.json
77 ./data_7/airlines-1560012550556.json


In [47]:
df_tweets, df_users = tables(df_objects)

In [48]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [49]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [50]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 8

In [51]:
files = path_files('./data_8/')

FileNotFoundError: [Errno 2] No such file or directory: './data_8/'

In [ ]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

In [ ]:
df_tweets, df_users = tables(df_objects)

In [ ]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [ ]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [ ]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 9

In [ ]:
files = path_files('./data_9/')

In [ ]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

In [ ]:
df_tweets, df_users = tables(df_objects)

In [ ]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [ ]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [ ]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

### Data Folder 10

In [ ]:
files = path_files('./data_10/')

In [ ]:
df_objects = pd.concat([read_json(i) for i in files], sort=False, ignore_index=True)

In [ ]:
df_tweets, df_users = tables(df_objects)

In [ ]:
df_tweets_clean, df_users_clean = select(df_tweets, df_users)

In [ ]:
df_tweets_clean.to_sql('TWEETS', conn_tweets, if_exists="append", index=False)

In [ ]:
df_users_clean.to_sql('USERS', conn_tweets, if_exists="append", index=False)

---

In [52]:
conn_tweets.close()


---


In [53]:
stop = datetime.now()

In [55]:
calculate_runtime(start, stop)

'Runtime of Loading process for 78 files was: 0 h, 47 m 3 s'